In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
broken = pd.read_csv('broken.csv', encoding='cp949')
june = pd.read_csv('june.csv', encoding='cp949')

In [3]:
glass = pd.read_csv('glass.csv')

In [11]:
glass.groupby('Type').count()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
Type,,,,,,,,,
1,70,70,70,70,70,70,70,70,70
2,76,76,76,76,76,76,76,76,76
3,17,17,17,17,17,17,17,17,17
5,13,13,13,13,13,13,13,13,13
6,9,9,9,9,9,9,9,9,9
7,29,29,29,29,29,29,29,29,29


In [3]:
broken['등록일시_date'] = pd.to_datetime(broken['등록일시']).dt.date

In [4]:
start = datetime.strptime('2022-06-01','%Y-%m-%d').date()
end = datetime.strptime('2022-06-30','%Y-%m-%d').date()
broken = broken[(broken['등록일시_date']>=start)&(broken['등록일시_date']<=end)]

In [5]:
broken_uniq = broken.drop_duplicates(subset = ['자전거번호', '등록일시'])

In [6]:
broken_uniq.describe()

,자전거번호,등록일시,고장구분,등록일시_date
count,15607,15607,15607,15607
unique,11250,11630,6,29
top,SPB-51383,2022-06-24 18:05,기타,2022-06-20
freq,9,6,4602,766


In [9]:
broken_uniq.reset_index(drop=True)

,자전거번호,등록일시,고장구분,등록일시_date
0,SPB-34424,2022-06-01 0:01,단말기,2022-06-01
1,SPB-62861,2022-06-01 0:03,타이어,2022-06-01
2,SPB-44585,2022-06-01 0:07,안장,2022-06-01
3,SPB-84526,2022-06-01 0:08,기타,2022-06-01
4,SPB-33405,2022-06-01 0:09,단말기,2022-06-01
...,...,...,...,...
15602,SPB-56997,2022-06-29 21:18,타이어,2022-06-29
15603,SPB-53871,2022-06-29 22:02,체인,2022-06-29
15604,SPB-57521,2022-06-29 22:04,안장,2022-06-29
15605,SPB-60598,2022-06-29 22:28,단말기,2022-06-29


In [10]:
df_inter = pd.merge(june, broken_uniq, on='자전거번호', how='inner')

In [13]:
len(june['반납대여소번호'].unique())

539

In [69]:
broken_uniq[broken_uniq['자전거번호']=='SPB-54065']

,자전거번호,등록일시,고장구분,등록일시_date
68054,SPB-54065,2022-06-17 13:58,타이어,2022-06-17


In [68]:
june[june['자전거번호']=='SPB-54065'].sort_values(['대여일시'])

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리
349326,SPB-54065,2022-06-03 16:56,4575,대림동 e편한세상영등포아델포레,0,2022-06-03 17:19,281,신동아아파트 앞,0,22,2842.84
915917,SPB-54065,2022-06-03 18:14,4566,대림성원상떼빌아파트,0,2022-06-03 18:45,702,목4동주민센터 옆,0,30,4866.38
969877,SPB-54065,2022-06-03 19:00,702,목4동주민센터 옆,0,2022-06-03 19:05,735,영도초등학교,0,4,608.92
984394,SPB-54065,2022-06-03 20:07,735,영도초등학교,0,2022-06-03 20:14,744,신목동역 2번 출구,0,7,1318.02
960446,SPB-54065,2022-06-03 20:48,744,신목동역 2번 출구,0,2022-06-03 22:10,731,서울시 도로환경관리센터,0,82,3897.94
329174,SPB-54065,2022-06-04 0:31,1974,신도림2차e편한세상아파트 203동 앞,0,2022-06-04 0:56,274,영등포역지하쇼핑센타 5번출구,0,25,2581.81
351882,SPB-54065,2022-06-04 0:59,274,영등포역지하쇼핑센타 5번출구,0,2022-06-04 1:03,283,아크로타워 스퀘어(영등포시장),0,3,656.24
334080,SPB-54065,2022-06-04 11:23,263,근로자회관 사거리,0,2022-06-04 11:36,275,신동아아파트,0,13,816.15
263466,SPB-54065,2022-06-04 11:47,275,신동아아파트,0,2022-06-04 11:56,245,삼성생명 당산사옥 앞,0,8,1448.88
334093,SPB-54065,2022-06-04 13:06,245,삼성생명 당산사옥 앞,0,2022-06-04 13:15,275,신동아아파트,0,8,1385.15


In [12]:
df_inter.describe()

,대여 대여소번호,대여거치대,반납대여소번호,반납거치대,이용시간,이용거리
count,518100.000000,518100.000000,518100.000000,518100.000000,518100.000000,518100.000000
mean,1455.945856,0.999745,427.264063,0.994968,23.768664,2796.222322
std,1553.853305,9.898256,199.637664,9.874820,27.438714,3376.616809
min,3.000000,0.000000,3.000000,0.000000,0.000000,0.000000
25%,338.000000,0.000000,247.000000,0.000000,7.000000,887.360000
50%,633.000000,0.000000,421.000000,0.000000,13.000000,1620.000000
75%,3010.000000,0.000000,592.000000,0.000000,31.000000,3272.380000
max,9999.000000,99.000000,770.000000,99.000000,1142.000000,88177.900000


In [14]:
df = df_inter.sort_values(['자전거번호','반납일시','등록일시']).iloc[:,[0,5,6,7,11]]

In [16]:
df.reset_index(drop=True)

,자전거번호,반납일시,반납대여소번호,반납대여소명,등록일시
0,SPB-30014,2022-06-09 19:29,209,유진투자증권빌딩 앞,2022-06-03 21:33
1,SPB-30014,2022-06-09 20:47,509,이마트 버스정류소 옆,2022-06-03 21:33
2,SPB-30014,2022-06-09 21:13,525,한양대병원사거리,2022-06-03 21:33
3,SPB-30014,2022-06-10 17:00,764,목동청소년수련관,2022-06-03 21:33
4,SPB-30014,2022-06-10 22:31,726,목동3단지 시내버스정류장,2022-06-03 21:33
...,...,...,...,...,...
518095,SPB-85000,2022-06-19 13:55,431,청계천 영도교,2022-06-07 19:16
518096,SPB-85000,2022-06-19 14:37,431,청계천 영도교,2022-06-07 19:16
518097,SPB-85000,2022-06-19 15:41,431,청계천 영도교,2022-06-07 19:16
518098,SPB-85000,2022-06-19 17:21,431,청계천 영도교,2022-06-07 19:16


In [17]:
df[df['반납대여소번호']==422]

,자전거번호,반납일시,반납대여소번호,반납대여소명,등록일시
64487,SPB-30304,2022-06-18 10:38,422,문화비축기지,2022-06-13 13:51
257868,SPB-30391,2022-06-24 8:53,422,문화비축기지,2022-06-18 13:47
151901,SPB-30419,2022-06-04 17:30,422,문화비축기지,2022-06-22 18:38
10932,SPB-30972,2022-06-01 7:31,422,문화비축기지,2022-06-20 7:32
233184,SPB-30980,2022-06-04 20:46,422,문화비축기지,2022-06-19 21:46
...,...,...,...,...,...
322095,SPB-63385,2022-06-03 18:11,422,문화비축기지,2022-06-08 7:47
187125,SPB-63446,2022-06-22 13:11,422,문화비축기지,2022-06-26 15:46
122063,SPB-63474,2022-06-20 7:22,422,문화비축기지,2022-06-02 12:57
122064,SPB-63474,2022-06-20 7:22,422,문화비축기지,2022-06-29 7:45


In [18]:
df['반납일시'] = pd.to_datetime(df['반납일시'])
df['등록일시'] = pd.to_datetime(df['등록일시'])

In [19]:
# 신고 이전 반납이력만 남기기
df_norm = df[(df['반납일시']<=df['등록일시'])]

In [20]:
df_final = df_norm.loc[df_norm.groupby(['자전거번호','등록일시'])['반납일시'].idxmax()]

In [22]:
df_final = df_final.reset_index(drop=True)

In [71]:
df_final[df_final['반납대여소번호']==422]

,자전거번호,반납일시,반납대여소번호,반납대여소명,등록일시
9162,SPB-54065,2022-06-17 09:38:00,422,문화비축기지,2022-06-17 13:58:00


In [25]:
df_final

,자전거번호,반납일시,반납대여소번호,반납대여소명,등록일시
0,SPB-30018,2022-06-01 13:45:00,286,우성아파트 교차로,2022-06-07 08:41:00
1,SPB-30018,2022-06-11 18:09:00,103,망원역 2번출구 앞,2022-06-11 18:56:00
2,SPB-30031,2022-06-11 21:47:00,119,서강나루 공원,2022-06-11 22:15:00
3,SPB-30052,2022-06-08 20:07:00,259,대방역6번출구,2022-06-10 09:35:00
4,SPB-30052,2022-06-12 17:16:00,680,꿈마루어린이도서관 앞,2022-06-16 17:15:00
...,...,...,...,...,...
13399,SPB-84931,2022-06-15 13:28:00,519,양지사거리,2022-06-26 14:31:00
13400,SPB-84963,2022-06-01 01:01:00,361,동묘앞역 1번출구 뒤,2022-06-06 13:23:00
13401,SPB-84964,2022-06-13 13:22:00,573,구의문주차장 앞,2022-06-13 14:35:00
13402,SPB-84979,2022-06-13 15:44:00,577,광진청소년수련관,2022-06-24 09:09:00


In [26]:
df_final.groupby(['반납대여소번호']).count().iloc[:,0]

반납대여소번호
3        2
102     45
103     30
104     14
105     13
      ... 
766     28
767      2
768     27
769     63
770    118
Name: 자전거번호, Length: 528, dtype: int64

In [27]:
df_count = pd.DataFrame(df_final.groupby(['반납대여소번호']).count().iloc[:,0])

In [28]:
df_count.columns=['고장횟수']

In [29]:
df_count

,고장횟수
반납대여소번호,
3,2
102,45
103,30
104,14
105,13
...,...
766,28
767,2
768,27


In [236]:
df_count.to_excel('반납대여소별_고장횟수.xlsx')

In [30]:
time = pd.read_excel('반납대여소별_고장횟수.xlsx')
name = pd.read_csv('3. 공공자전거 대여소 정보(22.06월 기준).csv', encoding='cp949')

In [31]:
name = name.iloc[4:,[0,1,2,3,4,5]]
name.head()

,대여소\n번호,보관소(대여소)명,소재지(위치),Unnamed: 3,Unnamed: 4,Unnamed: 5
4,102.0,망원역 1번출구 앞,마포구,서울특별시 마포구 월드컵로 72,37.5556488,126.9106293
5,103.0,망원역 2번출구 앞,마포구,서울특별시 마포구 월드컵로 79,37.55495071,126.9108353
6,104.0,합정역 1번출구 앞,마포구,서울특별시 마포구 양화로 59,37.55062866,126.9149857
7,105.0,합정역 5번출구 앞,마포구,서울특별시 마포구 양화로 48,37.55000687,126.9148254
8,106.0,합정역 7번출구 앞,마포구,서울특별시 마포구 독막로 4,37.54864502,126.9128265


In [32]:
name = name.rename(columns={'대여소\n번호':'반납대여소번호'})
name['반납대여소번호'] = name['반납대여소번호'].astype(int)

In [33]:
name['반납대여소번호'] = name['반납대여소번호'].astype(int)

In [34]:
name.iloc[565,:]

반납대여소번호                         802
보관소(대여소)명              한강진역 2번 출구 앞
소재지(위치)                         용산구
Unnamed: 3    서울특별시 용산구 한남동 726-494
Unnamed: 4              37.54115295
Unnamed: 5              127.0022125
Name: 569, dtype: object

In [40]:
df_inter = pd.merge(time, name, on='반납대여소번호', how='inner')

In [41]:
name.groupby('소재지(위치)').count()

,반납대여소번호,보관소(대여소)명,주소,위도,경도
소재지(위치),,,,,
강남구,159,159,159,159,159
강동구,119,119,119,119,119
강북구,58,58,58,58,58
강서구,184,184,184,184,184
관악구,76,76,76,76,76
광진구,86,86,86,86,86
구로구,104,104,104,104,104
금천구,74,74,74,74,74
노원구,127,127,127,127,127


In [42]:
df_inter.to_excel('반납대여소별_고장횟수_최종.xlsx')

In [38]:
name = name.rename(columns={'Unnamed: 3':'주소','Unnamed: 4':'위도','Unnamed: 5':'경도'})

In [39]:
name

,반납대여소번호,보관소(대여소)명,소재지(위치),주소,위도,경도
4,102,망원역 1번출구 앞,마포구,서울특별시 마포구 월드컵로 72,37.5556488,126.9106293
5,103,망원역 2번출구 앞,마포구,서울특별시 마포구 월드컵로 79,37.55495071,126.9108353
6,104,합정역 1번출구 앞,마포구,서울특별시 마포구 양화로 59,37.55062866,126.9149857
7,105,합정역 5번출구 앞,마포구,서울특별시 마포구 양화로 48,37.55000687,126.9148254
8,106,합정역 7번출구 앞,마포구,서울특별시 마포구 독막로 4,37.54864502,126.9128265
...,...,...,...,...,...,...
2652,5851,도림천 건널목 위,영등포구,영등포구 대림동 646-1,37.50208282,126.8931885
2653,5852,대림동현대2차 201동 앞,영등포구,영등포구 대림동 640-2,37.50300598,126.8938294
2654,5853,여의도역2번출구 앞,영등포구,여의도동 의사당대로 88,37.52260971,126.9230118
2655,5854,신한금융투자 앞,영등포구,여의대로 70,37.52508926,126.9240799


In [43]:
df_inter

,반납대여소번호,고장횟수,보관소(대여소)명,소재지(위치),주소,위도,경도
0,102,45,망원역 1번출구 앞,마포구,서울특별시 마포구 월드컵로 72,37.5556488,126.9106293
1,103,30,망원역 2번출구 앞,마포구,서울특별시 마포구 월드컵로 79,37.55495071,126.9108353
2,104,14,합정역 1번출구 앞,마포구,서울특별시 마포구 양화로 59,37.55062866,126.9149857
3,105,13,합정역 5번출구 앞,마포구,서울특별시 마포구 양화로 48,37.55000687,126.9148254
4,106,49,합정역 7번출구 앞,마포구,서울특별시 마포구 독막로 4,37.54864502,126.9128265
...,...,...,...,...,...,...,...
520,766,28,신목동역 3번출구,양천구,양천구 목동 136-1,37.54394913,126.883461
521,767,2,신정숲속마을아파트,양천구,양천구 신정로 110,37.50574493,126.8363113
522,768,27,목동5단지아파트 501동,양천구,양천구 목동,37.53566742,126.8815842
523,769,63,CBS방송국 앞,양천구,양천구 목동 917-1,37.52785492,126.8746719


In [44]:
df3 = pd.read_excel('반납대여소별_고장횟수_최종.xlsx')

In [45]:
df3['소재지(위치)'].unique()

array(['마포구', '서대문구', '영등포구', '동작구', '중구', '종로구', '광진구', '성동구', '동대문구',
       '양천구'], dtype=object)

In [46]:
df3.groupby(['소재지(위치)']).describe()

Unnamed: 0                                                        \
             count        mean         std    min     25%    50%     75%   
소재지(위치)                                                                    
광진구           31.0  365.935484   24.132876  322.0  355.50  364.0  384.50   
동대문구          65.0  434.000000   18.907670  402.0  418.00  434.0  450.00   
동작구            1.0  112.000000         NaN  112.0  112.00  112.0  112.00   
마포구           84.0  148.214286  123.017588    0.0   25.50   76.0  259.25   
서대문구          38.0   49.736842   20.042771   12.0   32.25   54.5   63.75   
성동구           49.0  358.693878   22.450134  326.0  340.00  352.0  376.00   
양천구           58.0  495.500000   16.886879  467.0  481.25  495.5  509.75   
영등포구          85.0  123.635294   25.086444   81.0  102.00  124.0  145.00   
종로구           60.0  220.583333   40.945723  168.0  191.75  208.5  236.25   
중구            54.0  249.722222   46.829444  167.0  220.25  252.5  297.75   

               반납대여소번호              ...         위도               경도  \
           max   count        mean  ...        75%        max count   
소재지(위치)                             ...                               
광진구      400.0    31.0  553.451613  ...  37.549400  37.571255  31.0   
동대문구     466.0    65.0  641.369231  ...  37.586594  37.603622  65.0   
동작구      112.0     1.0  237.000000  ...  37.496513  37.496513   1.0   
마포구      321.0    84.0  282.809524  ...  37.573982  37.587524  84.0   
서대문구      79.0    38.0  159.368421  ...  37.578686  37.594330  38.0   
성동구      401.0    49.0  544.530612  ...  37.563511  37.571640  49.0   
양천구      524.0    58.0  736.448276  ...  37.535396  37.550732  58.0   
영등포구     166.0    85.0  250.682353  ...  37.528099  37.544666  85.0   
종로구      294.0    60.0  371.916667  ...  37.579348  37.603992  60.0   
중구       311.0    54.0  409.277778  ...  37.566280  37.571014  54.0   

                                                                               \
               mean       std         min         25%         50%         75%   
소재지(위치)                                                                         
광진구      127.083292  0.010871  127.067192  127.074566  127.082245  127.090939   
동대문구     127.052046  0.014181  127.023376  127.042587  127.053535  127.063026   
동작구      126.914803       NaN  126.914803  126.914803  126.914803  126.914803   
마포구      126.914778  0.023352  126.879822  126.894464  126.912720  126.936356   
서대문구     126.930477  0.017314  126.902969  126.915548  126.929001  126.941181   
성동구      127.042190  0.015487  127.014061  127.030243  127.043800  127.056656   
양천구      126.857845  0.018340  126.825401  126.840164  126.863617  126.875101   
영등포구     126.909231  0.014132  126.885582  126.897827  126.905739  126.921333   
종로구      126.988519  0.016906  126.957832  126.974512  126.987675  127.001654   
중구       126.991656  0.016245  126.965408  126.977974  126.988098  127.006395   

                     
                max  
소재지(위치)              
광진구      127.106133  
동대문구     127.074310  
동작구      126.914803  
마포구      126.956688  
서대문구     126.967277  
성동구      127.073410  
양천구      126.886612  
영등포구     126.937790  
종로구      127.022850  
중구       127.023933  

[10 rows x 40 columns]

In [47]:
df3['반납대여소번호'] = df3['반납대여소번호'].astype(str)

In [48]:
df3

,Unnamed: 0,반납대여소번호,고장횟수,보관소(대여소)명,소재지(위치),주소,위도,경도
0,0,102,45,망원역 1번출구 앞,마포구,서울특별시 마포구 월드컵로 72,37.555649,126.910629
1,1,103,30,망원역 2번출구 앞,마포구,서울특별시 마포구 월드컵로 79,37.554951,126.910835
2,2,104,14,합정역 1번출구 앞,마포구,서울특별시 마포구 양화로 59,37.550629,126.914986
3,3,105,13,합정역 5번출구 앞,마포구,서울특별시 마포구 양화로 48,37.550007,126.914825
4,4,106,49,합정역 7번출구 앞,마포구,서울특별시 마포구 독막로 4,37.548645,126.912826
...,...,...,...,...,...,...,...,...
520,520,766,28,신목동역 3번출구,양천구,양천구 목동 136-1,37.543949,126.883461
521,521,767,2,신정숲속마을아파트,양천구,양천구 신정로 110,37.505745,126.836311
522,522,768,27,목동5단지아파트 501동,양천구,양천구 목동,37.535667,126.881584
523,523,769,63,CBS방송국 앞,양천구,양천구 목동 917-1,37.527855,126.874672


In [49]:
df3.groupby(['소재지(위치)']).describe()

Unnamed: 0                                                        \
             count        mean         std    min     25%    50%     75%   
소재지(위치)                                                                    
광진구           31.0  365.935484   24.132876  322.0  355.50  364.0  384.50   
동대문구          65.0  434.000000   18.907670  402.0  418.00  434.0  450.00   
동작구            1.0  112.000000         NaN  112.0  112.00  112.0  112.00   
마포구           84.0  148.214286  123.017588    0.0   25.50   76.0  259.25   
서대문구          38.0   49.736842   20.042771   12.0   32.25   54.5   63.75   
성동구           49.0  358.693878   22.450134  326.0  340.00  352.0  376.00   
양천구           58.0  495.500000   16.886879  467.0  481.25  495.5  509.75   
영등포구          85.0  123.635294   25.086444   81.0  102.00  124.0  145.00   
종로구           60.0  220.583333   40.945723  168.0  191.75  208.5  236.25   
중구            54.0  249.722222   46.829444  167.0  220.25  252.5  297.75   

                고장횟수             ...         위도               경도              \
           max count       mean  ...        75%        max count        mean   
소재지(위치)                          ...                                           
광진구      400.0  31.0  53.354839  ...  37.549400  37.571255  31.0  127.083292   
동대문구     466.0  65.0  24.307692  ...  37.586594  37.603622  65.0  127.052046   
동작구      112.0   1.0  71.000000  ...  37.496513  37.496513   1.0  126.914803   
마포구      321.0  84.0  19.369048  ...  37.573982  37.587524  84.0  126.914778   
서대문구      79.0  38.0  14.500000  ...  37.578686  37.594330  38.0  126.930477   
성동구      401.0  49.0  31.591837  ...  37.563511  37.571640  49.0  127.042190   
양천구      524.0  58.0  33.500000  ...  37.535396  37.550732  58.0  126.857845   
영등포구     166.0  85.0  35.094118  ...  37.528099  37.544666  85.0  126.909231   
종로구      294.0  60.0  13.183333  ...  37.579348  37.603992  60.0  126.988519   
중구       311.0  54.0  11.037037  ...  37.566280  37.571014  54.0  126.991656   

                                                                               
              std         min         25%         50%         75%         max  
소재지(위치)                                                                        
광진구      0.010871  127.067192  127.074566  127.082245  127.090939  127.106133  
동대문구     0.014181  127.023376  127.042587  127.053535  127.063026  127.074310  
동작구           NaN  126.914803  126.914803  126.914803  126.914803  126.914803  
마포구      0.023352  126.879822  126.894464  126.912720  126.936356  126.956688  
서대문구     0.017314  126.902969  126.915548  126.929001  126.941181  126.967277  
성동구      0.015487  127.014061  127.030243  127.043800  127.056656  127.073410  
양천구      0.018340  126.825401  126.840164  126.863617  126.875101  126.886612  
영등포구     0.014132  126.885582  126.897827  126.905739  126.921333  126.937790  
종로구      0.016906  126.957832  126.974512  126.987675  127.001654  127.022850  
중구       0.016245  126.965408  126.977974  126.988098  127.006395  127.023933  

[10 rows x 32 columns]

In [44]:
# 동대문구 9명, 종로구: 9명, 중구:7명